In [49]:
import umap
import numpy
import scipy as sp
import networkx as nx
from pprint import pprint as pprint

In [106]:
G = nx.DiGraph()

idx = 20
with open('graph.desc', 'r') as fh:
    for line_number, line in enumerate(fh):
        if idx>0 and line_number>2:
            article_name = line.split("\t")[1][1:][:-2]
            line_number = int(line.split("\t")[0])
            G.add_node(line_number, article_name=article_name)
            idx=idx-1
print (G.nodes)


NodeView((1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20))


In [107]:
pprint(nx.info(G))

('Name: \n'
 'Type: DiGraph\n'
 'Number of nodes: 20\n'
 'Number of edges: 0\n'
 'Average in degree:   0.0000\n'
 'Average out degree:   0.0000')
